In [2]:
import os
import sys
import json
import torch
from tqdm import tqdm
from typing import List
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
import peft

/home/dshome/anaconda3/envs/auto_gptq_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
int(os.environ.get("WORLD_SIZE", 1))

1

In [ ]:
base_model_path = 'models/'
adapter_path = 'models/'
tockenizer_path = 'models/'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(tockenizer_path, use_fast=False) #, padding_side='left')
# generation_config = GenerationConfig.from_pretrained(model_name, do_sample=True)

In [ ]:
# config = PeftConfig.from_pretrained(base_model_path)
base_model_config = AutoConfig.from_pretrained(base_model_path)
torch_dtype = base_model_config.torch_dtype

In [ ]:
quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch_dtype,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    ),

In [ ]:
generation_config = GenerationConfig.from_pretrained(base_model_path,
                                                     do_sample=True)

In [ ]:
# config = PeftConfig.from_pretrained(adapter_path)
# base_model_config = AutoConfig.from_pretrained(base_model_path)

# torch_dtype = base_model_config.torch_dtype

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    load_in_8bit=True,
    # quantization_config=quantization_config, # tесли не хотим 4-bit.
    torch_dtype=torch_dtype,
    device_map="auto",
)

In [ ]:
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    adapter_name="adapter1_name",
    torch_dtype=torch_dtype
)

In [ ]:
model.eval()

In [ ]:
print(model.active_adapter)

In [ ]:
# Настраиваем конфиги для адаптера.

In [ ]:
# Надо сделать так чтобы разморозились веса
# только у адарптера и обучался именно он, а не модель.

In [4]:
f'{int(torch.cuda.mem_get_info()[0] / 1024 ** 3) - 2}GB'

'8GB'

In [ ]:
# У меня сегодня тоже возникла задача решить проблему с нехваткой памяти (уже правда в рамках обучения другой нейросети), поэтому полез настраивать device_map и max_memory опции, чтобы тренировка, если места мало, могла залезать в системную оперативную память, вот как сделал:

model = model_types[model_type].from_pretrained(
  model_name,
  load_in_8bit=True,
  device_map='auto',
  max_memory={0: f'{int(torch.cuda.mem_get_info()[0] / 1024 ** 3) - 2}GB'},
  torch_dtype=torch_dtype,
  use_flash_attention_2=use_flash_attention_2
)